 ### Introduction to Data Science - 2019.2

 ### Valter Moreno

 ### Homework 2: Predicting Schools Performance

 ### Data

 The data for this project was provided in four csv files containing information on shools in São Paulo metropolitan region. They included characteristics of the students and classes, academic perfomance metrics, and results in the national evaluation exam (ENEM).

In [0]:
import numpy as np
import pandas as pd

pd.set_option('display.float_format', lambda x: '%.2f' % x)


In [0]:
# Reading the data into dataframes

enem = pd.read_csv('Data/ENEM2015.csv', header=0, names=['CD_ESCOLA','PARTICIPANTES','ENEM'], encoding='utf-8')
rendimento = pd.read_csv('Data/RendimentoEscolar2000-2015.csv', encoding='utf-8')
escolas = pd.read_csv('Data/DadosEscolares1996-2015.csv', encoding='utf-8')

censo = pd.read_csv('Data/ESC2013_RMSP_CEM.csv', encoding='utf-8')
censo.replace(['999,9', '999,99', '9999,99'], '', inplace=True)


In [0]:
enem.head()


In [0]:
enem.dtypes


In [0]:
censo.head()


In [0]:
print('Columns of the string type:')
print(censo.select_dtypes('object').columns.to_list())
print()
print('Columns of numeric type:')
print(censo.select_dtypes('number').columns.to_list())


In [0]:
rendimento.head()


In [0]:
print('Columns of the string type:')
print(rendimento.select_dtypes('object').columns.to_list())
print()
print('Columns of numeric type:')
print(rendimento.select_dtypes('number').columns.to_list())


 The last column of the dataframe is ill-formatted.
 Its contents will be fixed and stored as integers.

In [0]:
cols = list(rendimento.columns)
del cols[-1]
cols.append('EJATOT')
rendimento.columns = cols
rendimento.EJATOT = pd.to_numeric(rendimento.EJATOT.str.replace(',', ''))

rendimento.head()


In [0]:
escolas.head()


In [0]:
print('Columns of the string type:')
print(escolas.select_dtypes('object').columns.to_list())
print()
print('Columns of numeric type:')
print(escolas.select_dtypes('number').columns.to_list())


 ### Checking if the number of schools match in all files

In [0]:
cd_enem = enem.CD_ESCOLA.unique()
cd_censo = censo.CODESC.unique()
cd_rendimento = rendimento.CODMEC.unique()
cd_escolas = escolas.CODMEC.unique()

print('Number of records in each dataframe:')
print('cd_enem:', len(cd_enem))
print('cd_censo:', len(cd_censo))
print('cd_rendimento:', len(cd_rendimento))
print('cd_escolas:', len(cd_escolas), '\n')

print('Number of schools in the ENEM dataframe:')
print('cd_enem:', len(cd_enem))
print('cd_censo:', np.in1d(cd_censo, cd_enem).sum())
print('cd_rendimento:', np.in1d(cd_rendimento, cd_enem).sum())
print('cd_escolas:', np.in1d(cd_escolas, cd_enem).sum())


 Many of the schools that are in the ENEM2015.csv file are not in
 in the other datasets. Schools that are not in the ENEM file
 will be dropped then from the other dataframes. Only schools that
 are listed in the four dataframes will be used in the analysis.

In [0]:
cd_enem = set(cd_enem)
cd_censo = set(cd_censo)
cd_rendimento = set(cd_rendimento)
cd_escolas = set(cd_escolas)

cods = cd_enem.intersection(cd_censo, cd_rendimento, cd_escolas)

enem = enem[enem.CD_ESCOLA.isin(cods)]
censo = censo[censo.CODESC.isin(cods)]
rendimento = rendimento[rendimento.CODMEC.isin(cods)]
escolas = escolas[escolas.CODMEC.isin(cods)]


 Dataframes 'rendimento' and 'escolas' contain longitudinal data.
 I will get the number of schools for which there is data for each
 year in the time horizon.

In [0]:
def long_count(df, df_name):
      print('Dataframe:', df_name)
      for year in np.sort(df.ANO.unique()):
            print(year, ': ',
                  df[(df.ANO == year) & (df.CODMEC.isin(cods))].shape[0])
      print('\n')

long_count(rendimento, 'Rendimento')
long_count(escolas, 'Escolas')


 I decided to retain years for which there is data for at least 500 schools. This is
 necessary to allow meus to train and test the model with samples of reasonable size.
 Thus, I will be able to keep only the records for 2015 in 'rendimentos', and for
 2012, 2013, 2014 and 2015 in 'escolas'.

In [0]:

rendimento = rendimento[(rendimento.ANO == 2015) & 
                        (rendimento.CODMEC.isin(cods))]

rendimento[(rendimento.ANO == 2015) & (rendimento.CODMEC.isin(cods))].shape

escolas = escolas[(escolas.ANO.isin([2012, 2013, 2014, 2015])) &
                  (escolas.CODMEC.isin(cods))]

print('Number of remaining records in each dataframe:')
print('Enem:', enem.shape[0])
print('Censo:', censo.shape[0])
print('Rendimento:', rendimento.shape[0])
print('Escolas:', escolas.shape[0], '\n')

censo.to_csv('Data/censo.csv', index=False)
rendimento.to_csv('Data/rendimento.csv', index=False)
escolas.to_csv('Data/escolas.csv', index=False)


 ## Missing values
 I will performa a quick inspection of the dataframes for missing values.

In [0]:

def miss_data(df):
    cols = []
    values = []
    for col in df.columns:
        miss = df[col].isnull().sum()
        if miss > 0:
            cols.append(col)
            values.append(miss)
    missing = {'Columns': cols, 'Missing': values}
    return missing 


In [0]:
censo_miss = pd.DataFrame(miss_data(censo))
censo_miss['Percent missing'] = censo_miss.Missing/censo.shape[0]
print('Columns with missing values in dataframe Censo:', '\n',
      censo_miss.sort_values(by='Missing', ascending=False))


In [0]:
rendimento_miss = pd.DataFrame(miss_data(rendimento))
rendimento_miss['Percent missing'] = rendimento_miss.Missing/rendimento.shape[0]
print('Columns with missing values in dataframe Rendimento:', '\n',
      rendimento_miss.sort_values(by='Missing', ascending=False))


In [0]:
escolas_miss = pd.DataFrame(miss_data(escolas))
escolas_miss['Percent missing'] = escolas_miss.Missing/escolas.shape[0]
print('Columns with missing values in dataframe Escolas:', '\n',
      escolas_miss.sort_values(by='Missing',ascending=False))


In [0]:
censo_miss.to_csv('Data/censo_miss.csv', index=False)
rendimento_miss.to_csv('Data/rendimento_miss.csv', index=False)
escolas_miss.to_csv('Data/escolas_miss.csv', index=False)


 Alghough the 'rendimento' dataframe has no missing values, 'censo' and
 'escolas' do.
 Instead of inputing the missing values, I will drop the records with missing
 values for any of the features. Nevertheless, given the small size of the sample
 (505 schools), I want to retain as much data as possible.
 I start by discarding columns with more than 10% of missing data. Before,
 I do this, I will merge the 'enem', 'censo', and 'rendimento' dataframes, as
 they have data only for one year. Because the 'escolas' dataframe has longitudinal data,
 it will be treated separetely.

In [0]:
schools = enem.merge(censo, left_on = 'CD_ESCOLA', right_on='CODESC')
schools = schools.merge(rendimento, left_on = 'CD_ESCOLA', right_on='CODMEC')
schools.drop(['CODESC', 'CODMEC', 'ANO', 'DEP', 'NOME', 'SETEDU',
              'DISTRITO', 'SUBPREF', 'DIRET', 'ZONA',],
             axis=1, inplace=True)


In [0]:
def drop_miss(df, df_miss, percent):
      drop_cols = df_miss[df_miss['Percent missing'] >= percent].Columns
      df.drop(drop_cols, axis=1, inplace=True)
      return df


In [0]:
schools_miss = pd.DataFrame(miss_data(schools))
schools_miss['Percent missing'] = schools_miss.Missing/schools.shape[0]

schools = drop_miss(schools, schools_miss, .10)


In [0]:
escolas = escolas[escolas.CODMEC.isin(schools.CD_ESCOLA)]

escolas_miss = pd.DataFrame(miss_data(escolas))
escolas_miss['Percent missing'] = escolas_miss.Missing/escolas.shape[0]
escolas = drop_miss(escolas, escolas_miss, .10)


 Here are the remaining columns with missing values:

In [0]:
schools_miss = pd.DataFrame(miss_data(schools))
schools_miss['Percent missing'] = schools_miss.Missing/schools.shape[0]
print('Columns with missing values in dataframe Schools:', '\n',
      schools_miss.sort_values(by='Missing', ascending=False))
print()
print('Sorted by name:', '\n',
      schools_miss.sort_values(by='Columns'))


In [0]:
escolas_miss = pd.DataFrame(miss_data(escolas))
escolas_miss['Percent missing'] = escolas_miss.Missing/escolas.shape[0]
print('Columns with missing values in dataframe Escolas:', '\n',
      escolas_miss.sort_values(by='Missing', ascending=False))
print()
print('Sorted by name:', '\n',
      escolas_miss.sort_values(by='Columns'))


 Based on the descriptions of the variables in the data dictionary,
 features that seem less relevant to predict the school's result
 in ENEM and that have missing values will be removed from the
 dataframes.

 The following columns will be dropped from the 'schools' dataframe:
     - AP9EF_09
     - AB1EM_10
     - APR3EM_11
     - AP9EF_11
     - AP9EF_10
     - AP3EM_12

In [0]:
schools.drop(['AP9EF_09', 'AB1EM_10', 'APR3EM_11',
              'AP9EF_11', 'AP9EF_10', 'AP3EM_12'],
             axis =1, inplace=True)


 The following columns will be dropped from the 'escolas' dataframe:
     - 0A3
     - 0A4
     - 4A6
     - 5A6
     - >6
     - TotalEdInf
     - MENOR3
     - CLE9F1S
     - ALE9F1S
     - CLE9F2S
     - ALE9F2S
     - CLE9F5S
     - ALE9F5S
     - CLE9F3S
     - ALE9F3S
     - CLE9F4S
     - ALE9F4S
     - CLE9F9S
     - ALE9F9S
     - ALE9F8S
     - CLE9F8S
     - CLE9F6S
     - ALE9F6S
     - CLE9F7S
     - ALE9F7S

In [0]:
escolas.drop(escolas_miss[escolas_miss['Percent missing'] > 0.01]['Columns'], 
             axis=1, inplace=True)


 The 'escolas' dataframe is in the long format. It will be converted to the
 wide format before the rows with missing values are removed.

In [0]:
escolas.drop(['TIPOESC', 'NOME', 'DEP', 'SETEDU', 'DISTRITO',
              'SUBPREF', 'CORED', 'ZONA'], axis=1, inplace=True)

escolas_long = pd.melt(escolas, id_vars=['CODMEC','ANO'], 
                       var_name='Vars', value_name='Values')

escolas_long['NewVar'] = escolas_long['Vars'] + '_' + escolas_long['ANO'].map(str)
      
escolas_long.drop(['ANO', 'Vars'], axis=1, inplace=True)

escolas = escolas_long.pivot(index='CODMEC', columns='NewVar',
                             values='Values').reset_index()


 Finally, I will remove rows with missing values from both dataframes, and
 merge them into a new 'schools' dataframe. The new dataframe will be recorded
 to the 'Schools.csv' file.

In [0]:
schools.dropna(inplace=True)
escolas.dropna(inplace=True)

schools = schools.merge(escolas, left_on='CD_ESCOLA', right_on='CODMEC')

schools.to_csv('Data/Schools.csv', index=False)      

schools.head()


 The final dataset contains data for 436 schools.

In [0]:
print(schools.shape)
schools.describe()